In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import seaborn as sns
from IPython.display import display

#### Functions

In [ ]:
def try_cast_to_int_else_string(value):
    try:
        # Try to convert to integer
        return int(value)
    except ValueError:
        # If conversion fails, return as string
        return str(value)

## Prepping data

In [ ]:
pizza = pd.read_csv("data/pizza_sales_with_data_quality_issues.csv")
pizza["order_date"] = pd.to_datetime(pizza["order_date"], format="%Y-%m-%d")
# Apply the function to the 'order_id' column
pizza["order_id"] = pizza["order_id"].apply(try_cast_to_int_else_string)

## 1) Double trouble

Duplicated data in clean datasets refers to the repetition of the same information, like listing "The Hitchhiker's Guide to the Galaxy" multiple times in a book collection. This can lead to errors, such as overcounting items or making decisions based on incorrect information. Removing duplicates is crucial for accurate analysis and reliable results.



#### Solution

In [ ]:
# Creating DataFrame
df = pd.DataFrame(pizza.copy())

# Finding duplicates
duplicates = df.duplicated()

# Showing duplicates
display(df[duplicates])

# If you want to keep the first occurrence and remove the rest of the duplicates
df_1 = df.drop_duplicates()

## 2) Is pizza always on time?

Outlying order dates in datasets are a common issue encountered during data analysis, especially in fields involving time-series or sequential data. These outliers can arise from various sources, such as data entry errors, system glitches, or incorrect data processing. They may manifest as dates that are significantly earlier or later than the rest of the data, or even as completely nonsensical dates (e.g., dates in the future for a supposedly historical dataset or dates well before the actual start of the dataset's timeframe). Identifying and addressing these outliers is crucial, as they can skew analysis, lead to incorrect conclusions, and affect model performance.


Now, let's delve into examining outlying order dates in our dataset to understand their impact and decide on appropriate corrective actions. This process involves detecting these anomalies, assessing their nature, and determining the best course of action—whether it's correcting, removing, or otherwise accounting for these outliers in our analysis.



In [ ]:
pizza.head()

#### Solution

In [ ]:
df = pizza.copy()
# Summarize orders by decade
df_decade_summary = (
    df.groupby(df["order_date"].dt.year // 10 * 10)
    .size()
    .reset_index(name="orders_count")
)
df_decade_summary

In [ ]:
df["decade"] = (df["order_date"].dt.year // 10) * 10
# Count of orders per decade
orders_per_decade = df.groupby("decade")["order_id"].nunique()

# Plotting
plt.figure(figsize=(8, 5))
orders_per_decade.plot(kind="bar")
plt.title("Number of Orders per Decade")
plt.xlabel("Decade")
plt.ylabel("Number of Orders")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.tight_layout()
plt.show()

In [ ]:
# Filtering the data to exclude the 2010 decade
orders_except_2010 = df[df["decade"] != 2010].groupby("decade")["order_id"].nunique()

# Plotting
plt.figure(figsize=(8, 5))
orders_except_2010.plot(kind="bar")
plt.title("Number of Orders per Decade (Excluding 2010s)")
plt.xlabel("Decade")
plt.ylabel("Number of Orders")
plt.xticks(rotation=45)
plt.grid(axis="y")
plt.tight_layout()
plt.show()

#### Conclusion 

In [ ]:
# Remove outliers (assuming orders not in the decade of 2010s as outliers for the context of this example)
df_2 = df[df["order_date"].dt.year // 10 * 10 == 2010]

## 3) Misshapen values
* Looks like there's something wrong with the order_id column. Shall we investigate?
* Moreover, it's worth checking if there are rows containing empty values. How to check that? Can we safely drop them?

#### Solution

In [ ]:
df = pizza.copy()

# Convert all 'order_id' to string to ensure consistent processing
df["order_id_str"] = df["order_id"].apply(str)

# Calculate the length of each 'order_id'
df["length"] = df["order_id_str"].apply(len)

# Group lengths exceeding 5 into a single category, ensuring all categories are strings
df["length_group"] = df["length"].apply(lambda x: str(x) if x <= 10 else "10+")

# Count occurrences of each length category
length_counts = df["length_group"].value_counts().sort_index()

length_counts

In [ ]:
# Filtering rows where 'order_id' has more than 2 characters
df_3 = df[df["length"] > 10]
display(df_3)

In [ ]:
df_4 = df[df.isnull().any(axis=1)]
df_4

## 4) Outlier detection with IQR method

In the most general sense, an outlier is a data point which differs significantly from other observations. 

A boxplot tells us, more or less, about the distribution of the data. It gives a sense of how much the data is actually spread out, what its range is and its skewness. A boxplot enables us to draw inferences from it for ordered data. It tells us about the various metrics of  data arranged in ascending order.

It labels the minimum on one side, maximum on the other and a median in the middle. That would mean:

* Minimum is the minimum value in the data set.
* Maximum is the maximum value in the data set.

So, the difference between the two tells us about the range of a data set.


* The median is the median (or center point), also called second quartile, of the data resulting from the fact that the data is ordered.
* Q1 is the first quartile of the data, which is to say 25 percent of the data lies between minimum and Q1.
* Q3 is the third quartile of the data, which is to say 75 percent of the data lies between minimum and Q3.

To detect the outliers using this method, we define a new range - let’s call it the decision range. Any data point lying outside this range is considered an outlier and is accordingly dealt with. The range is as given below:

Lower Bound: (Q1 - 1.5 * IQR)
Upper Bound: (Q3 + 1.5 * IQR)


The 1.5 IQR rule determines that any data point that’s 1.5 IQR points below the first quartile of data or above the third quartile is an outlier.

#### Visualisation

In [ ]:
# Using Plotly to create a boxplot for total price
fig = px.box(df, y="total_price", title="Total price boxplot")

# Adjust the layout to make the boxplot taller
fig.update_layout(height=500)  # Adjust the height as needed
fig.show()

#### Solution
Try and code your own function to detect outliers in 'total_price' column according to IQR method.

In [ ]:
# Function to detect outliers using the IQR method
def detect_outliers_with_iqr(df, column_name):
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[column_name] < lower_bound) | (df[column_name] > upper_bound)]
    return outliers


# Detect outliers in the total_price column
outliers = detect_outliers_with_iqr(df, "total_price")
outliers